In [ ]:
import os
import torch
import joblib
import torchvision.transforms as transforms

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
model_name = 'resnet50'

path_1 = os.path.join('..', 'fine_tuned_models', model_name, 'resnet50_featExtractor.pt')
path_2 = os.path.join('..', 'fine_tuned_models', model_name, 'best_Regression_model.pth')
scaler_path = os.path.join('..', 'fine_tuned_models', model_name, 'scaler.pkl')

feat_extractor = torch.load(path_1, map_location=device)
income_predictor = torch.load(path_2, map_location=device)
scaler = joblib.load(scaler_path)

feat_extractor.eval()
income_predictor.eval()

In [ ]:
def extract_features_and_predict_income(image_path, feature_extractor, income_pred):

    # Define transformations to preprocess the input image
    transform = transforms.Compose([
        transforms.Resize((299, 299)),  # Resize to the input size of the model
        transforms.CenterCrop(299),
        transforms.ToTensor(),           # Convert image to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    ])

    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0)  # Add a batch dimension
    input_tensor = input_tensor.to(device)

    # Extract features
    with torch.no_grad():
        features = feature_extractor(input_tensor)
    features = features.cpu().numpy().flatten()  # Convert to numpy array and flatten

    # Convert features to tensor and pass through regression model
    features_tensor = torch.tensor(features, dtype=torch.float32).unsqueeze(0).to(device)

    with torch.no_grad():
        scaled_income_prediction = income_pred(features_tensor).cpu().numpy().flatten()
    income_prediction = scaler.inverse_transform(scaled_income_prediction.reshape(-1, 1)).flatten()

    return income_prediction[0]

In [ ]:
extract_features_and_predict_income("24.6503_68.501.png", feat_extractor, income_predictor)